<a href="https://colab.research.google.com/github/luguzman/NLP/blob/main/Transformer_para_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar las dependencias

**Paper original**: All you need is Attention https://arxiv.org/pdf/1706.03762.pdf

In [3]:
import numpy as np
import math
import re
import time
from google.colab import drive

In [4]:
# Le preguntamos a google colab por la version 2. más reciente
try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Fase 2: Pre Procesado de Datos



## Carga de Ficheros

Importamos los ficheros de nuestro Google Drive personal

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
with open("/content/drive/MyDrive/Curso NLP/Transformer/data/europarl-v7.es-en.en", 
          mode = "r", encoding = "utf-8") as f:
    europarl_en = f.read()
with open("/content/drive/MyDrive/Curso NLP/Transformer/data/europarl-v7.es-en.es", 
          mode = "r", encoding = "utf-8") as f:
    europarl_es = f.read()
with open("/content/drive/MyDrive/Curso NLP/Transformer/data/P85-Non-Breaking-Prefix.en", 
          mode = "r", encoding = "utf-8") as f:
    non_breaking_prefix_en = f.read()
with open("/content/drive/MyDrive/Curso NLP/Transformer/data/nonbreaking_prefix.es", 
          mode = "r", encoding = "utf-8") as f:
    non_breaking_prefix_es = f.read()

In [ ]:
europarl_en[:100]

'Resumption of the session\nI declare resumed the session of the European Parliament adjourned on Frid'

In [ ]:
europarl_es[:100]

'Reanudación del período de sesiones\nDeclaro reanudado el período de sesiones del Parlamento Europeo,'

## Limpieza de datos

Vamos a obtener los non_breaking_prefixes como una lista de palabras limpias con un punto al final para que nos sea más fácil de utilizar. Estos non_breaking_prefixes son palabras/letras que después de un punto no termina la oración como i.e (es decir) o inclusive no nos cuesta añadir las letras para un mejor performance. 

In [ ]:
non_breaking_prefix_en

'a\nb\nc\nd\ne\nf\ng\nh\ni\nj\nk\nl\nm\nn\no\np\nq\nr\ns\nt\nu\nv\nw\nx\ny\nz\nmessrs\nmlle\nmme\nmr\nmrs\nms\nph\nprof\nsr\nst\na.m\np.m\nvs\ni.e\ne.g'

In [ ]:
non_breaking_prefix_en = non_breaking_prefix_en.split("\n")
non_breaking_prefix_en = [' ' + pref + '.' for pref in non_breaking_prefix_en]
non_breaking_prefix_es = non_breaking_prefix_es.split("\n")
non_breaking_prefix_es = [' ' + pref + '.' for pref in non_breaking_prefix_es]

Necesitaremos cada palabra y otro símbolo que queramos mantener en minúsculas y separados por espacios para que podamos "tokenizarlos".

In [ ]:
corpus_en = europarl_en
# Añadimos $$$ después de los puntos de frases sin fin
for prefix in non_breaking_prefix_en:
    corpus_en = corpus_en.replace(prefix, prefix + '$$$')
# Sustituimos con expresiones regulares: cualquier punto que le sigue o un texto o un número o una letra 
# por ".$$$", ie buscaremos los puntos que están en medio de palabras y que luego luego sin ningun espacio
# contengan un número o una letra. Usamos "\." para especificar que busque un punto ya que en re el punto
# significa "cualquier cosa". El "?=" indica que lo que sigue de la expresión regular no debe ser reemplazado,
# por ejemplo 10 a.m sería 10 a.$$$m
corpus_en = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_en)
# Eliminamos los marcadores ".$$$"
corpus_en = re.sub(r"\.\$\$\$", '', corpus_en)
# Eliminamos espacios múltiples
corpus_en = re.sub(r"  +", " ", corpus_en)
corpus_en = corpus_en.split('\n')

corpus_es = europarl_es
for prefix in non_breaking_prefix_es:
    corpus_es = corpus_es.replace(prefix, prefix + '$$$')
corpus_es = re.sub(r"\.(?=[0-9]|[a-z]|[A-Z])", ".$$$", corpus_es)
corpus_es = re.sub(r"\.\$\$\$", '', corpus_es)
corpus_es = re.sub(r"  +", " ", corpus_es)
corpus_es = corpus_es.split('\n')

## Tokenizar el Texto

In [ ]:
# Codificamos/tokenizamos el corpus en inglés e indicamos un tamaño suficiente de palabras a considerar 
# del vocabulario
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_en, target_vocab_size=2**13)
# Codificamos/tokenizamos el corpus en español
tokenizer_es = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    corpus_es, target_vocab_size=2**13)

In [ ]:
# Le preguntamos al tokenizer por el tamaño del vocabulario y ñadimos 2:
# Un token que indicará inicio de frase y el otro el final de la frase
VOCAB_SIZE_EN = tokenizer_en.vocab_size + 2 # = 8198
VOCAB_SIZE_ES = tokenizer_es.vocab_size + 2 # = 8225

In [ ]:
# Dado que el token que agregamos para inicio de la frase ocupara la pisición -2 del VOCAB_SIZE_EN y el de
# fianl de la frase -1 del VOCAB_SIZE_EN lo que estamos haciendo para cada oración en corpus_en colocar
# el token de inicio de frase + la frase codificada + el token de final de la frase 
inputs = [[VOCAB_SIZE_EN-2] + tokenizer_en.encode(sentence) + [VOCAB_SIZE_EN-1]
          for sentence in corpus_en]
outputs = [[VOCAB_SIZE_ES-2] + tokenizer_es.encode(sentence) + [VOCAB_SIZE_ES-1]
           for sentence in corpus_es]

## Eliminamos las frases demasiado largas

In [ ]:
# Dado que no tenemos una computadora tan potente o un servidor externo con más potencia a lo que nos
# ofrece google colab removeremos todas las frases que tengan mas de 20 carácteres.
MAX_LENGTH = 20
# Lo que hacemos es recorrer todas las frases y si su longitud es > 20 se queda con el indice de la frase
# para posteriormente eliminar las frases
idx_to_remove = [count for count, sent in enumerate(inputs)
                 if len(sent) > MAX_LENGTH]
# Observemos que eliminamos los indices del ultimo al primero, ya que si borramos los primeros al eliminar una 
# frase las demás frases estarían cambiando de posición
for idx in reversed(idx_to_remove):
    del inputs[idx]   # Eliminamos las frases del corpus en ingles
    del outputs[idx]  # Eliminamos las frases del corpus en español también ya que si no nos quemos con un corpus más grande

# De la misma manero hacemos lo mismo para el corpues en español
idx_to_remove = [count for count, sent in enumerate(outputs)
                 if len(sent) > MAX_LENGTH]
for idx in reversed(idx_to_remove):
    del inputs[idx]
    del outputs[idx]

In [ ]:
# Creamos un backup por si la sesión de google colab nos saca
pd.DataFrame(inputs).to_csv("/content/drive/MyDrive/Curso NLP/Transformer/data/inputs.csv", index = False)
pd.DataFrame(outputs).to_csv("/content/drive/MyDrive/Curso NLP/Transformer/data/outputs.csv", index = False)

## Creamos las entradas y las salidas

A medida que entrenamos con bloques, necesitaremos que cada entrada tenga la misma longitud. Rellenamos con el token apropiado, y nos aseguraremos de que este token de relleno no interfiera con nuestro entrenamiento más adelante.

In [ ]:
# Hacemos padding en todas las entradas y salidas (frases) para que tengan la misma longitud. 
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=MAX_LENGTH)
outputs = tf.keras.preprocessing.sequence.pad_sequences(outputs,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=MAX_LENGTH)

In [ ]:
# Creamos un dataset mezclando, haciendo un shuffle y todos los ultimos arreglos antes de que se creen
# finalmente los conjuntos de entrenamiento.

# Podemos jugar con los siguiente hiperparámetros.
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# Juntamos inputs y outputs en una variable
dataset = tf.data.Dataset.from_tensor_slices((inputs, outputs))

# Mejoramos el modo en que se almacenan los datos mientras se encuentra en la fase de entrenamiento. Esto no
# el perfomance del modelo pero si que incrementará la velocidad de entrenamiento y de acceso a los datos
dataset = dataset.cache()

# Usamos shuffle para indicar que queremos mezclar el dataset con un buffer de tamaño 20 con bloques de 64 
# en 64 (frases)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

# Aceleramos el proceso de los datos. Este prefetch de nuevo lo único que hace es que sirve para acceder a los
# datos más rápidamente. El parámetro AUTOTUNE es para ajustar la caché venga ya con los primeros
# bloques preparados para que el acceso a los datos cada vez no tenga que esperar al siguiente bloque, sino que ya
# tenga el siguiente bloque en memoria y por tanto aceleremos la fase de entrenamiento.
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# Fase 3: Construcción del Modelo

## Embedding

Fórmula de la Codificación Posicional:

$PE_{(pos,2i)} =\sin(pos/10000^{2i/dmodel})$

$PE_{(pos,2i+1)} =\cos(pos/10000^{2i/dmodel})$

In [ ]:
class PositionalEncoding(layers.Layer):

    def __init__(self):
        super(PositionalEncoding, self).__init__()
    # Creamos el metodo get_angle el cual se encargara de calcular el angulo del seno y coseno, ie, los valores de las funciones expresados arriba.
    # pos (es un tensor con posiciones de 0-19 debido a MAX_LENGTH = 19): posición que ocupa la palabra dentro de la frase
    # i (vector o array de todas las dimensiones posibles del espacio de emdedding): la dimensión en la que estamos embebiendo
    # d_model: la dimension total del modleo del espacio de embeddings
    def get_angles(self, pos, i, d_model): # pos: (seq_length, 1) i: (1, d_model)
        angles = 1 / np.power(10000., (2*(i//2)) / np.float32(d_model))
        return pos * angles # (seq_length, d_model) ya que estamos haciendo una multiplicaciones matricial 
  
  # NOTA : El subíndice utilizado en las formulas es la notación matematica expresar un número par o impar que no es lo mismo a lo que estamos 
  # expresando en la potencia del denomidor de ambas fórmulas. Además usamos decimales ya que no queremos perder información valiosa
  # NOTA: (i//2) lo usamos para quedarnos con la parte entera asi si i = 10 o i = 11 su parte entera será 5, así podemos calcular el ángulo de la
  # posición par "x" y su consecuente.

    # Creamos el método que se va a utilizar cada vez que creemos cada una de estas capas para añadirla a la arquitectura global.
    # inputs: Número de entradas
    def call(self, inputs):
        seq_length = inputs.shape.as_list()[-2]   # Longitud de la secuencia, dado que quiero que sean números los pasamos a formato lista y nos quedamos con la penúltima entrada ay que es la que nos dice cual es la dimensión, ie, cuantas palabras.
        d_model = inputs.shape.as_list()[-1]      # Para acceder a las dimensiones del modelos
        angles = self.get_angles(np.arange(seq_length)[:, np.newaxis],    # np.newaxis: genera una dimensión vacia adicional. Pasamos de tener una lista a una matriz
                                 np.arange(d_model)[np.newaxis, :],       # Pasamos a tener un vector en fila
                                 d_model)
        
        # Corregimos las dimensiones columnas pares 
        angles[:, 0::2] = np.sin(angles[:, 0::2]) # Indicamos que la secuencia empieza en cero y ":2" es el llamdo stride que significa el salto es de 2 en 2. Por lo que accederíamos a las posiciones pares 
        # Corregimos las dimension columnas impares
        angles[:, 1::2] = np.cos(angles[:, 1::2])
        # Tomamos el angles que ya habíamos generado anteriormente pero le agregaremos una dimención vacia adicional por delante ya que es la
        # que corresponde con los batches, con los bloques de entrenamiento. La 2° y 3° dimension corresponderá a los bloques (angles) generados
        pos_encoding = angles[np.newaxis, ...]
        return inputs + tf.cast(pos_encoding, tf.float32) # Devolvemos los inputs + el posicional encoding que acabos de generar

## Attention

### Cálculo de la Atención

$Attention(Q, K, V ) = \text{softmax}\left(\dfrac{QK^T}{\sqrt{d_k}}\right)V $

In [ ]:
# queries: "Q"
# keys: "K"
# values: "V"
# mask: parámetro que podrá ser la máscara de lookahead que no permitirá que el descodificador vea palabras más allá del padding establecido o
# podrá ser absolutamente nada si queremos que acceda a toda la frase.
def scaled_dot_product_attention(queries, keys, values, mask):
    # Realizamos el producto matricial del númerador
    product = tf.matmul(queries, keys, transpose_b=True)
    
    # Declaramos la dimensión de las claves
    # tf.shape(keys)[-1]: es para quedarnos con el número de la última dimensión 
    keys_dim = tf.cast(tf.shape(keys)[-1], tf.float32)  #tf.float32: ya que queremos quedarnos con todos los decimales
    scaled_product = product / tf.math.sqrt(keys_dim)   # Dividimos entre todas y cada una de las entradas
    
    # De forma ocpional se puede aplicar una mascara (lookahead). Lo cual es anular o no cada una de las entradas 
    # Aplicaremos un pequeño truco y esto será multiplicar los valores donde esta la máscara (los unos basicamente), porque la mascara tiene
    # ceros y unos. Vamos a multiplicar los unos por -infinito, ie, un número mu pequeño ya que al resultado de esto posteriormente le aplicaré
    # la función softmax, función en la cual -infinito = 0 va creciendo de modo que en 0 vale 0.5 y sigue creciendo hasta +infinito = 1
    # De modo que no tengan impacto más adelante como parte del algoritmo. En python no se puede poner -inifinito y por eso usamos -1e9
    if mask is not None:
        scaled_product += (mask * -1e9)
    
    # Una vez aplicada la máscara, calculamos la atención como el producto de softmax aplicado a lo anterior por la matriz V
    attention = tf.matmul(tf.nn.softmax(scaled_product, axis=-1), values) # axis=-1: para que sea a cada una de la filas de  ultima dimensión tomada (observaciones de la Q que es la que tiene que sumar 1)
    
    #Nota: con esto tenemos nuestro producto escalar escalado
    return attention

### Sub capa de atención de encabezado múltiple

In [ ]:
class MultiHeadAttention(layers.Layer):
    
    # nb_proj: número de espacios en los que deseo proyectar
    def __init__(self, nb_proj):
        super(MultiHeadAttention, self).__init__()  # Inicializamos el objeto self de la clase MultiHeadAttention
        self.nb_proj = nb_proj
    
    # Todo lo que vaya a depender de los datos y vaya utilizar es mejor declararlo en build en lugar del init
    def build(self, input_shape):
        # Definimos la dimension
        self.d_model = input_shape[-1]  # La dimensión del embedding se encuentra en la última posición
        # realizamo una aserción en caso de no cumplirse dará error y mostrará en consola lo que ocurrió
        assert self.d_model % self.nb_proj == 0 
        
        # Número de projecciones en cada dimensión 
        self.d_proj = self.d_model // self.nb_proj
        
        # Definimos 3 capas densas. Por tanto tenemos una neurona para cada una de las dimensiones del espacio vectorial en cuestion.
        self.query_lin = layers.Dense(units=self.d_model)
        self.key_lin = layers.Dense(units=self.d_model)
        self.value_lin = layers.Dense(units=self.d_model)
        
        self.final_lin = layers.Dense(units=self.d_model)
        
    def split_proj(self, inputs, batch_size): # inputs: (batch_size, seq_length, d_model)
        # Definimos la dimensión de la tramnsformación 
        shape = (batch_size,    # mantenemos el propio tamaño del bloque
                 -1,            # basicamente será la longitud de la propio secuencia, que no se verá alterada para cada una de ellas
                 self.nb_proj,  # para cada una de ellas decido el número de proyecciones
                 self.d_proj)   # para cada proyección recibo la info en el espacio de la dimensión de proyecciones que hayamos calculado dinamicamente
        # redimensionas la entrada
        splited_inputs = tf.reshape(inputs, shape=shape) # (batch_size, seq_length, nb_proj, d_proj)
        return tf.transpose(splited_inputs, perm=[0, 2, 1, 3]) # (batch_size, nb_proj, seq_length, d_proj)
      # Ejemplo: si d_model = 15 y nb_proj = 3, pues basicamente me quedarían 3 submatrices con cada palabra proyectado en un espacio de dimensión 5
      # Return: lo que obtenemos es para cada bloque una de sus proyecciones [0], para cada proyección las palabras de la secuencia, para cada palabra de la 
      # secuencia el espacio lineal proyectado
    
    # 1.
    def call(self, queries, keys, values, mask):
      # El tamaño del batch siempre lo va a definir la Q, que es la que siempre voy a querer ir prediciendo. 
        batch_size = tf.shape(queries)[0] # primera dimensión de query indica el bloque
        
        # Queremos aplicar 3 transformaciones lineales; las Q, las K y las V
        # Declaramos 3 capas densas 
        queries = self.query_lin(queries)
        keys = self.key_lin(keys)
        values = self.value_lin(values)
        
        # Dividmos los queries, keys y values en cada uno de los espacios especificados en el metodo split_proj y vendrán en metodo adecuado
        # para palicarle la atención de producto escalar escalado
        queries = self.split_proj(queries, batch_size)
        keys = self.split_proj(keys, batch_size)
        values = self.split_proj(values, batch_size)
        
        # Calculamos la atención
        attention = scaled_dot_product_attention(queries, keys, values, mask)
    
        # recuperamos para cada bloque la sequencia de tantas proyecciones de dicho tamaño para que sean las dos últimas dimensiones las que
        # se junten para volver a juntarlo.
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        
        # Concatemos todo lo que habíamos separado
        concat_attention = tf.reshape(attention,
                                      shape=(batch_size, -1, self.d_model))
        # -1: mantemenos la longitud de la segunda dimensión 
        # self.d_model: que la tercera y última dimensión tiene que ser de tamaño d_model
        
        # Una última función lineal, una última capa densa será la que se encargará de juntar toda la información y proceder
        outputs = self.final_lin(concat_attention)
        
        return outputs

## Codificación

In [ ]:
# Objetivo crear una estructura de n capas
class EncoderLayer(layers.Layer):
    
    #Definimos el constructor
    # FFN_units: Número de unidades en la capa oculta
    #n_porj: vamos a pryectar a un espacio n dimensional al hacer el embedding.
    # dropout_rate: % de neuronas que no se van activar durante la fase de entrenamiento
    def __init__(self, FFN_units, nb_proj, dropout_rate):
        super(EncoderLayer, self).__init__()
        self.FFN_units = FFN_units
        self.nb_proj = nb_proj
        self.dropout_rate = dropout_rate
    
    # El siguiente metodo es llamado cada vez que manda a construir una capa, solo pasandole el tamaño de la entrada
    def build(self, input_shape):
        self.d_model = input_shape[-1] # tamaño de la dimensión
        
        self.multi_head_attention = MultiHeadAttention(self.nb_proj) # particiones de la dimensión
        self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)
        
        self.dense_1 = layers.Dense(units=self.FFN_units, activation="relu")
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_2 = layers.Dropout(rate=self.dropout_rate)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)
    
    # inputs: tamaño de la capa inmediata anterior
    # training: para denotar si estamos entrenando, para de ser así activar el dropout_out
    def call(self, inputs, mask, training):
        attention = self.multi_head_attention(inputs, # q
                                              inputs, # k
                                              inputs, # v 
                                              mask)
        attention = self.dropout_1(attention, training=training)
        attention = self.norm_1(attention + inputs)   # aplicamos la normalización a la suma de attention + las inputs
        
        # 2° fase: aplicamos 2 capas densas
        outputs = self.dense_1(attention)
        outputs = self.dense_2(outputs) # aplicamos esta capa ya que el resultado lo que queremos es una dimensión igual a d_model
        outputs = self.dropout_2(outputs, training=training)
        outputs = self.norm_2(outputs + attention)
        
        return outputs

In [ ]:
# 
class Encoder(layers.Layer):
    
    def __init__(self,
                 nb_layers, # Número de veces que queremos repetir la operación del EncoderLayer
                 FFN_units,
                 nb_proj,
                 dropout_rate,
                 vocab_size,  # Nos lo dará el propio tokenizador
                 d_model,
                 name="encoder"):
        super(Encoder, self).__init__(name=name)
        self.nb_layers = nb_layers
        self.d_model = d_model
        
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout_rate)
        self.enc_layers = [EncoderLayer(FFN_units,
                                        nb_proj,
                                        dropout_rate) 
                           for _ in range(nb_layers)]
    
    def call(self, inputs, mask, training):
        # Fase de codificación
        outputs = self.embedding(inputs) # la capa de embedding me proyecta al espacio n dimensional
        # En las capas de embedding multiplicamos los pesos por raíz de d_model. Se hace para el modelo sea más robusto, ya que al multplicar los
        # pesos por la raíz de d_model crecen un poco más y evitan problemas cercanos a cero para evitarnos tener problemas más adelante de convergencia
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # Realizamos el encoding posicional
        outputs = self.pos_encoding(outputs)
        outputs = self.dropout(outputs, training)
        
        # Aplicamos n veces la capa encoder. Sobrescribimos nb_layers veces las salidas 
        for i in range(self.nb_layers):
            outputs = self.enc_layers[i](outputs, mask, training)

        return outputs

## Descodificación

In [ ]:
class DecoderLayer(layers.Layer):
    
    def __init__(self, FFN_units, nb_proj, dropout_rate):
        super(DecoderLayer, self).__init__()
        self.FFN_units = FFN_units # No. de neuronas de la capa oculta
        self.nb_proj = nb_proj     # No. de proyecciones
        self.dropout_rate = dropout_rate  
    
    def build(self, input_shape):
        self.d_model = input_shape[-1]
        
        # Self multi head attention
        self.multi_head_attention_1 = MultiHeadAttention(self.nb_proj)
        self.dropout_1 = layers.Dropout(rate=self.dropout_rate)
        self.norm_1 = layers.LayerNormalization(epsilon=1e-6)
        
        # Multi head attention combinado con la salida del encoder 
        self.multi_head_attention_2 = MultiHeadAttention(self.nb_proj)
        self.dropout_2 = layers.Dropout(rate=self.dropout_rate)
        self.norm_2 = layers.LayerNormalization(epsilon=1e-6)
        
        # Feed foward
        self.dense_1 = layers.Dense(units=self.FFN_units,
                                    activation="relu")    # capa totalmente conectada con función relu para presindir de todos los valores negativos
        self.dense_2 = layers.Dense(units=self.d_model)
        self.dropout_3 = layers.Dropout(rate=self.dropout_rate)
        self.norm_3 = layers.LayerNormalization(epsilon=1e-6)
        
    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        # Aplicamos la 1° multihead attention layer de dos que aplicaremos
        attention = self.multi_head_attention_1(inputs, 
                                                inputs,
                                                inputs,
                                                mask_1)
        # Despues de algo chonho aplicamos un dropout
        attention = self.dropout_1(attention, training)
        # Normalizamos de acuerdo al paper
        attention = self.norm_1(attention + inputs)
        
        # Aplicamos la 2° multihead attention layer de dos que aplicaremos
        attention_2 = self.multi_head_attention_2(attention,
                                                  enc_outputs,
                                                  enc_outputs,
                                                  mask_2)
        attention_2 = self.dropout_2(attention_2, training)
        attention_2 = self.norm_2(attention_2 + attention)
        
        outputs = self.dense_1(attention_2)     # Aplicamos una primera capa densa al attention_2
        outputs = self.dense_2(outputs)         # una segunda a lo generado
        outputs = self.dropout_3(outputs, training)
        outputs = self.norm_3(outputs + attention_2)
        
        return outputs

In [ ]:
class Decoder(layers.Layer):
    
    def __init__(self,
                 nb_layers,     # Número de veces que queremos repetir la operación del DecoderLayer
                 FFN_units,
                 nb_proj,       # No de proyecciones 
                 dropout_rate,  
                 vocab_size,    
                 d_model,       # Dimensión del espacio vectorial que coincide con el no de columnas
                 name="decoder"):
        super(Decoder, self).__init__(name=name)
        self.d_model = d_model
        self.nb_layers = nb_layers
        
        self.embedding = layers.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding()
        self.dropout = layers.Dropout(rate=dropout_rate)
        
        self.dec_layers = [DecoderLayer(FFN_units,
                                        nb_proj,
                                        dropout_rate) 
                           for _ in range(nb_layers)]
    
    def call(self, inputs, enc_outputs, mask_1, mask_2, training):
        outputs = self.embedding(inputs)    # Utilizamos un capa de embedding para incrustar la entrada de vectores tokenizados a un espacio d-dimensional
        outputs *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # Tomamos el resultados y multiplicamos entrada a entrada por raiz cuadrada del resultado de castear d_model a un tensor float32 por recomendación del paper
        outputs = self.pos_encoding(outputs)  # Usamos el propio posicional encoder a la salida para poder tener relaciones únicas entre las coordenadas y las palabras
        outputs = self.dropout(outputs, training) # Con cierta proba no se activan ciertas neuronas
        
        # Sobrescribimos los outputs; creamos multiples descodificadores
        for i in range(self.nb_layers):
            outputs = self.dec_layers[i](outputs,
                                         enc_outputs,   # salida del decodificador
                                         mask_1,  
                                         mask_2,
                                         training)

        return outputs

## Transformer

In [ ]:
# Nota: Transformer no hereda de layer dado que este ya es un modelo y heredad de tf.keras.Model
class Transformer(tf.keras.Model):
    
    def __init__(self,
                 vocab_size_enc,    # Dimensiones del vocabulario del encoder
                 vocab_size_dec,    # Dimensiones del vocabulario del decoder (ya que dos vocabularios)
                 d_model,           # No de columnas (dimensión del espacio vectorial de embedding)
                 nb_layers,         # No. de capas. Cuantas veces utilizaremos las capas de encoder y decoder
                 FFN_units,         # No. de neuronas de la capa feed forward al final del encoder y del decoder
                 nb_proj,           # No. de proyecciones que deseamos que el multihead attention divida nuestro espacio vectorial
                 dropout_rate,      # El ratio de neuronas que queramos que no se activen durante la fase de entrenamiento.
                 name="transformer"):
        super(Transformer, self).__init__(name=name)
        
        self.encoder = Encoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout_rate,
                               vocab_size_enc,
                               d_model)
        self.decoder = Decoder(nb_layers,
                               FFN_units,
                               nb_proj,
                               dropout_rate,
                               vocab_size_dec,
                               d_model)
        self.last_linear = layers.Dense(units=vocab_size_dec, name="lin_ouput")
    
    # Creamos la mascara para el padding para los tokens a la hr de hacerle el padding
    def create_padding_mask(self, seq): #seq: (batch_size, seq_length) 
        mask = tf.cast(tf.math.equal(seq, 0), tf.float32)   # Tomamos la secuencia dodne la función tf.math.equal(seq,0) me pondrá un número 1 donde la secuencia tuviera un 0 y un 0 donde no hubiera un 0
        return mask[:, tf.newaxis, tf.newaxis, :] 
        # Esta mascará lo que haremos es expandirla porque ahora será una mascara con todo 0 o 1, con tantas filas como batch_size y tantas columnas como seq_length
        # mask[:, tf.newaxis, tf.newaxis, :] la primera se queda al inicio (batch_size), añado 2 nuevas dimensiones (tf.newaxis) y me quedo con los valores de cada secuencia al final (:)
        # Lo que estamos haciendo es agregar dos dimensiones esto para k,q,v y las segunda cada uno de los renglones ya que queremos que se aplique a toda una matriz en cuestion, ie, estamos redimensionando 

    # La función de la sig función será desactivar los elementos últimos, cuyos no queremos que aparezcan en la frase 
    # esta mascara debe tener unos en el tringulo superior 
    def create_look_ahead_mask(self, seq):
        seq_len = tf.shape(seq)[1]
        look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0) # creamos un matriz triangular inferior. Con -1,0 le decimos que no queremos que la columna j sea mayor que el renglon i
        return look_ahead_mask
        # Conclusión: Por lo tanto la mascara impide que conozca las palabras que va a haber en el futuro.
        # Notemos que no he añadido ningun tipo de padding por tanto pensaríamos que hace falta aplicar la función paddin_mask pero no nos hará falta del todo
        # gracias a la función tf.maximum
    
    def call(self, enc_inputs, dec_inputs, training):
        # Generamos la capa del encoder 
        enc_mask = self.create_padding_mask(enc_inputs)   # Solo nos preocupa que sea del tamaño adecuado 
        # Generamos la primera capa del decoder
        dec_mask_1 = tf.maximum(
            self.create_padding_mask(dec_inputs),
            self.create_look_ahead_mask(dec_inputs)
        )
        # Creamos la segunda capa del decoder
        dec_mask_2 = self.create_padding_mask(enc_inputs)
        
        
        enc_outputs = self.encoder(enc_inputs, enc_mask, training)
        dec_outputs = self.decoder(dec_inputs,
                                   enc_outputs,
                                   dec_mask_1,
                                   dec_mask_2,
                                   training)
        
        outputs = self.last_linear(dec_outputs)
        
        return outputs

In [1]:
# Ejemplo de como trabajan las siguientes funciones
def create_padding_mask( seq): #seq: (batch_size, seq_length) 
  mask = tf.cast(tf.math.equal(seq, 0), tf.float32)  
  return mask[:, tf.newaxis, tf.newaxis, :] 
    
def create_look_ahead_mask( seq):
  seq_len = tf.shape(seq)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0) # creamos un matriz triangular inferior. Con -1,0 le decimos que no queremos que la columna j sea mayor que el renglon i
  return look_ahead_mask


In [8]:
seq = tf.cast([[234,510,0,129,6,0,0,0]], tf.int32)
# Lo siguiente crea un tensor donde a la propia frase le ha añadido una dimensión y de hecho 2 dimensiones extra en medio para ser capaz de 
# recrear esa palabra en un espacio de dimensión superior. Espacio donde por cierto, la propia función por construcción como yo lo que hago 
# es buscar donde están los 0´s, me ha colocado 1´s donde había 0´s.
print(create_padding_mask(seq))
# Lo siguiente crea una matriz con número de filas y columnas igual a la longitud de la secuencia y donde me es imposible acceder a todo lo
# que viene después, ie a los númeor que están más allá de la diagonal
print(create_look_ahead_mask(seq))
# Lo siguiente hace que se me añadan los 0´s en cuestión y se me va a ampliar al tamaño adecuado pero al hacer el máximo entre este y el 
# create_look_ahead_mask me va aguardar los 1´s de ese triangulo superior cosa que no guardaba el padding. Podemos apreciar que ademas de que
# tengamos unos arriba de la diagonal también me bloquea ya donde hay 0´s
print(tf.maximum(create_padding_mask(seq), create_look_ahead_mask(seq)))

tf.Tensor([[[[0. 0. 1. 0. 0. 1. 1. 1.]]]], shape=(1, 1, 1, 8), dtype=float32)
tf.Tensor(
[[0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]], shape=(8, 8), dtype=float32)
tf.Tensor(
[[[[0. 1. 1. 1. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1. 1. 1. 1.]
   [0. 0. 1. 1. 1. 1. 1. 1.]
   [0. 0. 1. 0. 1. 1. 1. 1.]
   [0. 0. 1. 0. 0. 1. 1. 1.]
   [0. 0. 1. 0. 0. 1. 1. 1.]
   [0. 0. 1. 0. 0. 1. 1. 1.]
   [0. 0. 1. 0. 0. 1. 1. 1.]]]], shape=(1, 1, 8, 8), dtype=float32)


# Entrenamiento

In [ ]:
tf.keras.backend.clear_session()

# Hiper Parámetros
D_MODEL = 128 # 512
NB_LAYERS = 4 # 6
FFN_UNITS = 512 # 2048
NB_PROJ = 8 # 8
DROPOUT_RATE = 0.1 # 0.1

transformer = Transformer(vocab_size_enc=VOCAB_SIZE_EN,
                          vocab_size_dec=VOCAB_SIZE_ES,
                          d_model=D_MODEL,
                          nb_layers=NB_LAYERS,
                          FFN_units=FFN_UNITS,
                          nb_proj=NB_PROJ,
                          dropout_rate=DROPOUT_RATE)

In [ ]:
# Trabajaremos con SparseCategoricalCrossentropy ya que queremos maximizar la probabilidad de una palabra se encuentre a lado de otra
# y SparseCategoricalCrossentropy es el objeto standar cuando lo que tenemos son probabilidades y de hecho no son probas hasta que apliquemos softmax
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                            reduction="none")
# from_logits=True: definimos que las salidas son numero reales preparados para aplicarseles la función softmax 
# reduction: no queremos que aplique ninguna reducción, media, etc. No queremos un único número

def loss_function(target, pred):
    mask = tf.math.logical_not(tf.math.equal(target, 0)) # Buscamos las posiciones donde la matriz target tenga token iguales a 0, marcaremos los valores que no fueran 0´s
    loss_ = loss_object(target, pred) 
    
    mask = tf.cast(mask, dtype=loss_.dtype)   # para poder enmascarar, quitar esos lugares donde la pérdida no me interesa porque aporta 0 a lo que es el modelo
    loss_ *= mask # Filtramos que loss solo tiene valores no nulos 
    
    return tf.reduce_mean(loss_)

# Nos quedamos con la pérdida y la precisión en el entrenamiento
train_loss = tf.keras.metrics.Mean(name="train_loss")
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name="train_accuracy")

In [ ]:
# Optimizador del gradiente descendente
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)    # Formula que considerará los primeors 4000 pasos
        arg2 = step * (self.warmup_steps**-1.5) # Formula que considerará el learning rate después de los 4000 pasos
        
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)  # Por lo tanto el learning rate será variable

leaning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(leaning_rate,
                                     beta_1=0.9,
                                     beta_2=0.98,
                                     epsilon=1e-9)
        

In [ ]:
checkpoint_path = "/content/drive/MyDrive/Curso NLP/Transformer/checkpoints/"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)
# max_to_keep: máximo núemro de checkpoints a almacenar

# Si después de crear todos estos vinculos el manager detecta que ya hay un checkpoint en el sistema
# el manager se encargará de cargarlo
if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    print("Inicio del epoch {}".format(epoch+1))
    start = time.time()
    
    # Reseteamos train_loss y train_accuracy en cada epoch 
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    # batch: identificador de la fila; lote número 0, lote número 1, etc
    # enc_inputs: frase de inicio. Entrada del codificador
    # targets_ frase objetivo. Contra que voy a comparar la salida del descodificador.
    for (batch, (enc_inputs, targets)) in enumerate(dataset):
        dec_inputs = targets[:, :-1]      # El último es un token de fin de frase. Así lo construimos 
        dec_outputs_real = targets[:, 1:] # Todas las frases del bloque a excepción del primer token que es de inicio de frase
        with tf.GradientTape() as tape:   # Construimos un objeto que nos permitira registrar todo lo que le ocurra al modelo con respecto a las predicciones
            predictions = transformer(enc_inputs, dec_inputs, True) # True: dado que estamos en fase de entrenamiento
            loss = loss_function(dec_outputs_real, predictions) # Aplicamos la función de pérdidas
        
        # Gracias a que creamos el objeto tape podemos pedir que obtenga del objeto guardado el gradiente de 
        # la función lde pérdidas con respecto a las variables del transformer
        gradients = tape.gradient(loss, transformer.trainable_variables)
        # Aplicamos los gradientes de nuestro optimizador respecto a las varibales que se pueden modificar/entrenar
        # Es decir le decimos cuales son las varibales más culpables de que estemos comentiendo ese error
        optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
        
        train_loss(loss)    # Evaluamos la pérdida
        train_accuracy(dec_outputs_real, predictions) # Evaluamos la presición
        
        # Si el bloque es multiplo de 50 mostramos la siguiente info para llevar registro
        if batch % 50 == 0:
            print("Epoch {} Lote {} Pérdida {:.4f} Precisión {:.4f}".format(
                epoch+1, batch, train_loss.result(), train_accuracy.result()))
    
    # No queremos iniciar otra epoca hasta que hallamos guardado todos los pesos de la época inmediata 
    # anterior, por lo tanto haremos un checkpoint
    ckpt_save_path = ckpt_manager.save()
    print("Guardando checkpoint para el epoch {} en {}".format(epoch+1,
                                                        ckpt_save_path))
    print("Tiempo que ha tardado 1 epoch: {} segs\n".format(time.time() - start))

Inicio del epoch 1
Epoch 1 Lote 0 Pérdida 5.7712 Precisión 0.0000
Epoch 1 Lote 50 Pérdida 6.0703 Precisión 0.0008
Epoch 1 Lote 100 Pérdida 6.0209 Precisión 0.0249
Epoch 1 Lote 150 Pérdida 5.9672 Precisión 0.0341
Epoch 1 Lote 200 Pérdida 5.8880 Precisión 0.0388
Epoch 1 Lote 250 Pérdida 5.7829 Precisión 0.0431
Epoch 1 Lote 300 Pérdida 5.6780 Precisión 0.0512
Epoch 1 Lote 350 Pérdida 5.5599 Precisión 0.0575
Epoch 1 Lote 400 Pérdida 5.4499 Precisión 0.0622
Epoch 1 Lote 450 Pérdida 5.3427 Precisión 0.0662
Epoch 1 Lote 500 Pérdida 5.2409 Precisión 0.0704
Epoch 1 Lote 550 Pérdida 5.1469 Precisión 0.0750
Epoch 1 Lote 600 Pérdida 5.0560 Precisión 0.0796
Epoch 1 Lote 650 Pérdida 4.9704 Precisión 0.0841
Epoch 1 Lote 700 Pérdida 4.8874 Precisión 0.0882
Epoch 1 Lote 750 Pérdida 4.8114 Precisión 0.0924
Epoch 1 Lote 800 Pérdida 4.7401 Precisión 0.0967
Epoch 1 Lote 850 Pérdida 4.6730 Precisión 0.1008
Epoch 1 Lote 900 Pérdida 4.6113 Precisión 0.1048
Epoch 1 Lote 950 Pérdida 4.5494 Precisión 0.1086
Epoc

# Evaluación

In [ ]:
# Esta función traduce frases en inglés a tokens en castellano
def evaluate(inp_sentence):
  # Tokenizamos la frase
    inp_sentence = \
        [VOCAB_SIZE_EN-2] + tokenizer_en.encode(inp_sentence) + [VOCAB_SIZE_EN-1]
    # añadimos un eje en la posición 0, ie, añadimos uan dimensión a la frase esto porque solo es una frase
    # y habíamos estado entrenando con lotes de frases
    enc_input = tf.expand_dims(inp_sentence, axis=0)
    
    # Añadimos una dimensión igualmente al output para que sea un lote de predicciones 
    output = tf.expand_dims([VOCAB_SIZE_ES-2], axis=0)
    
    # Habrá que hacer varias iteraciones de nuestro transformer pues recordemos que a cada iteración, solo
    # nos devuelve la siguiente palabra. Recordemos que ademas nosotros hemos definido un máximo de longitud
    # de la frase = 20 por lo que a lo mucho obtendremos 20 palabras
    for _ in range(MAX_LENGTH):
        # hacemos la predicción:
        predictions = transformer(enc_input, output, False) #(1, seq_length, VOCAB_SIZE_ES)
        # De todas las prediccipnes me quedo con predicitions para todos los lotes(:), ie un solo lote porque
        # solo hay uno me quedo con la última palabra predicha por el transformer (-1) y me quedo con todas las
        # palabras en castellano (:)
        prediction = predictions[:, -1:, :]
        # Obtenemos el indice de esa palabra más probable de todo VOCAB_SIZE_ES
        predicted_id = tf.cast(tf.argmax(prediction, axis=-1), tf.int32)
        
        # si el tokenizador considera que es final de frase pues hemos finalizado
        if predicted_id == VOCAB_SIZE_ES-1:
            return tf.squeeze(output, axis=0) # para que el resultado sea un tensor unidimensional
        # En otro caso ir agregando las palabras predichas
        output = tf.concat([output, predicted_id], axis=-1)
        
    # por si la frase era demasiodo largo y no vio nunca una palabra de final de frase aun así convertimos
    # lo analizado hasta MAX_LENGTH en un vector unidimensional    
    return tf.squeeze(output, axis=0)

In [ ]:
# Función que construirá ya la frase de salida en español
def translate(sentence):
    output = evaluate(sentence).numpy()
    
    # decodificamos los tokens que no son ni de inicio, ni final de frase
    predicted_sentence = tokenizer_es.decode(
        [i for i in output if i < VOCAB_SIZE_ES-2]
    )
    
    print("Entrada: {}".format(sentence))
    print("Traducción predicha: {}".format(predicted_sentence))

In [ ]:
translate("This is a problem we have to solve.")

Entrada: This is a problem we have to solve.
Traducción predicha: Es un problema que debemos resolver.


In [ ]:
translate("This is a really powerful tool!")

Entrada: This is a really powerful tool!
Traducción predicha: ¡Es una hermosa hermosa!


In [ ]:
translate("This is an interesting course about Natural Language Processing")

Entrada: This is an interesting course about Natural Language Processing
Traducción predicha: Es un documento de trabajo interesante sobre el procedimiento de idioma Naígena
